<a href="https://www.kaggle.com/code/ayushkhaire/top-1000-cryptos-daily-updates?scriptVersionId=185905680" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import requests as rq
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
from kaggle_secrets import UserSecretsClient
from datetime import datetime
import json
import subprocess
import logging
import time

# Yeasterday data :)

In [2]:
df = pd.read_csv('/kaggle/input/top-1000-cryptos-historical/1000_cryptos.csv')
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2021-07-28,0.129788,0.129788,0.075315,0.088785,0.088785,7549110.0,BTC-USD
1,1,2021-07-29,0.088938,0.089018,0.059275,0.061926,0.061926,883695.0,BTC-USD
2,2,2021-07-30,0.062761,0.062771,0.048437,0.056137,0.056137,388272.0,BTC-USD
3,3,2021-07-31,0.056141,0.063744,0.053865,0.056623,0.056623,259941.0,BTC-USD
4,4,2021-08-01,0.056605,0.060404,0.048375,0.049528,0.049528,194323.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
1124143,1124143,2024-06-20,0.055313,0.057461,0.053745,0.054660,0.054660,22993943.0,SIS-USD
1124144,1124144,2024-06-21,0.054663,0.055493,0.052983,0.053846,0.053846,23106521.0,SIS-USD
1124145,1124145,2024-06-22,0.053847,0.054488,0.051402,0.052878,0.052878,20602543.0,SIS-USD
1124146,1124146,2024-06-23,0.052881,0.054707,0.051727,0.051971,0.051971,17310259.0,SIS-USD


# Define headers

In [3]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Extract symbols

In [4]:
 link = 'https://finance.yahoo.com/crypto/?offset=0&count=100'

# Get symbols and hit urls

In [5]:
resp_main_list = []
i = 0
for page in tqdm(range(10)):
    url = f'https://finance.yahoo.com/crypto/?offset={i*100}&count=100'
    r = rq.get(url,headers = headers)
    resp_main_list.append(r)
    i = i+1
#     print(url)

100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


In [6]:
symbol_list = []

for r in resp_main_list:
    soup = BeautifulSoup(r.text,'html.parser')
    symbs= soup.find_all('a',{'class':'Fw(600) C($linkColor)'})
    for s in symbs:
        symbol_list.append(s.text)
        
print(len(symbol_list))

1000


In [7]:
unique_symbol_list = []

for sy in symbol_list:
    if sy not in unique_symbol_list:
        unique_symbol_list.append(sy)

unique_symbol_list = list(set(unique_symbol_list))
print(len(unique_symbol_list))

998


# Now , get the link for today downloads from a tesla stock 

### This we willl use to use "code" by yahoo finance to figure out dates 

In [8]:
'''

link1 = https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1684241704&period2=1715864104&interval=1d&events=history&includeAdjustedClose=true

link2 = https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684241769&period2=1715864169&interval=1d&events=history&includeAdjustedClose=true


here the period values are exactly same.
'''

# After research , i got they are unix timestamps in period . lets try it out !

import time

# Get current Unix timestamp
current_timestamp = int(time.time())

date_str = "2015-01-01"
date_obj = datetime.strptime(date_str, "%Y-%m-%d")
period1 = int(time.mktime(date_obj.timetuple()))


period2 = current_timestamp  

period1, period2 = period2, period1

# Print the timestamps in the desired format
print(f"period1={period1}&period2={period2}")


period1=1719574636&period2=1420070400


# create stocks directory

In [9]:
os.mkdir('crypto_data')

# Download all files

In [10]:
for stock in tqdm(symbol_list[:1000]):
    url = f"https://query1.finance.yahoo.com/v7/finance/download/{stock}?period1={period2}&period2={period1}&interval=1d&events=history&includeAdjustedClose=true"
#     print(url)
    r = rq.get(url, headers=headers)
    if r.status_code == 200:
        with open(f"crypto_data/{stock}.csv", "wb") as file:
            file.write(r.content)
#         print(f"File downloaded successfully : {stock}.csv")
#     else:
#         print(f"Failed to download file : {stock}.csv")
#     time.sleep(3)


# https://query1.finance.yahoo.com/v7/finance/download/AMD?period1=1684244457&period2=1715866857&interval=1d&events=history&includeAdjustedClose=true

100%|██████████| 1000/1000 [01:46<00:00,  9.35it/s]


# Read a sample

In [11]:
first_file = os.listdir('/kaggle/working/crypto_data')[0]
print(first_file)
df = pd.read_csv(f'/kaggle/working/crypto_data/{first_file}')
df

DEL-USD.csv


,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-06,0.303964,0.307309,0.300579,0.301586,0.301586,167203
1,2021-10-07,0.301459,0.306809,0.298946,0.300621,0.300621,184094
2,2021-10-08,0.300632,0.302074,0.277019,0.277025,0.277025,164601
3,2021-10-09,0.277083,0.292689,0.270985,0.271936,0.271936,157797
4,2021-10-10,0.271939,0.273383,0.269467,0.272299,0.272299,168680
...,...,...,...,...,...,...,...
992,2024-06-24,0.002932,0.003042,0.002917,0.002950,0.002950,231031
993,2024-06-25,0.002950,0.003061,0.002933,0.002996,0.002996,195354
994,2024-06-26,0.002996,0.003020,0.002905,0.002936,0.002936,326457
995,2024-06-27,0.002936,0.002944,0.002804,0.002870,0.002870,422359


# Work on dataset

In [12]:
csvlist = os.listdir('/kaggle/working/crypto_data')

In [13]:
megadf = pd.DataFrame()

In [14]:
for sym, file in zip(symbol_list, csvlist):
    df = pd.read_csv(f'/kaggle/working/crypto_data/{file}')
    df['Symbol'] = sym
    megadf = pd.concat([megadf, df], ignore_index=True)

In [15]:
megadf

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2021-10-06,0.303964,0.307309,0.300579,0.301586,0.301586,167203.0,BTC-USD
1,2021-10-07,0.301459,0.306809,0.298946,0.300621,0.300621,184094.0,BTC-USD
2,2021-10-08,0.300632,0.302074,0.277019,0.277025,0.277025,164601.0,BTC-USD
3,2021-10-09,0.277083,0.292689,0.270985,0.271936,0.271936,157797.0,BTC-USD
4,2021-10-10,0.271939,0.273383,0.269467,0.272299,0.272299,168680.0,BTC-USD
...,...,...,...,...,...,...,...,...
1117199,2024-06-24,0.103063,0.103622,0.098507,0.102343,0.102343,1698542.0,SPX28081-USD
1117200,2024-06-25,0.102402,0.107269,0.102246,0.106932,0.106932,3938163.0,SPX28081-USD
1117201,2024-06-26,0.106932,0.112654,0.106932,0.109156,0.109156,6146774.0,SPX28081-USD
1117202,2024-06-27,0.109281,0.110327,0.105706,0.109225,0.109225,1567015.0,SPX28081-USD


In [16]:
first_date = megadf['Date'][0]
last_date = megadf['Date'].iloc[-1]

In [17]:
first_date

'2021-10-06'

In [18]:
megadf.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume       float64
Symbol        object
dtype: object

In [19]:
megadf['Date'] = pd.to_datetime(megadf['Date'], format="%Y-%m-%d")

In [20]:
megadf.dtypes

Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume              float64
Symbol               object
dtype: object

In [21]:
os.mkdir('dataset')

In [22]:
megadf.to_csv(f'dataset/1000_cryptos.csv')

# final dataset

In [23]:
df = pd.read_csv(f'/kaggle/working/dataset/1000_cryptos.csv')

In [24]:
df

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,0,2021-10-06,0.303964,0.307309,0.300579,0.301586,0.301586,167203.0,BTC-USD
1,1,2021-10-07,0.301459,0.306809,0.298946,0.300621,0.300621,184094.0,BTC-USD
2,2,2021-10-08,0.300632,0.302074,0.277019,0.277025,0.277025,164601.0,BTC-USD
3,3,2021-10-09,0.277083,0.292689,0.270985,0.271936,0.271936,157797.0,BTC-USD
4,4,2021-10-10,0.271939,0.273383,0.269467,0.272299,0.272299,168680.0,BTC-USD
...,...,...,...,...,...,...,...,...,...
1117199,1117199,2024-06-24,0.103063,0.103622,0.098507,0.102343,0.102343,1698542.0,SPX28081-USD
1117200,1117200,2024-06-25,0.102402,0.107269,0.102246,0.106932,0.106932,3938163.0,SPX28081-USD
1117201,1117201,2024-06-26,0.106932,0.112654,0.106932,0.109156,0.109156,6146774.0,SPX28081-USD
1117202,1117202,2024-06-27,0.109281,0.110327,0.105706,0.109225,0.109225,1567015.0,SPX28081-USD


# Upload Dataset

# User secrets

In [25]:
user_secrets = UserSecretsClient()
kaggle_apikey = user_secrets.get_secret("kaggle_apikey")
kaggle_username = user_secrets.get_secret("kaggle_username")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_apikey

# Metadata file

In [26]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print('Starting upload >>>>>>') 
print('Creating metadata file >>>>')
data = {
    "id": "ayushkhaire/top-1000-cryptos-historical"
}
metadata_file_location = '/kaggle/working/dataset/dataset-metadata.json' 
with open(metadata_file_location, 'w', encoding='utf-8') as metadata_file:
    json.dump(data, metadata_file)
print('Metadata file created')

Starting upload >>>>>>
Creating metadata file >>>>
Metadata file created


# upload

In [27]:
retries = 0
while retries < 5:
    try:
        command = "kaggle datasets version -p '/kaggle/working/dataset' -m 'Update' -r zip"
        subprocess.run(command, shell=True, check=True)
        logger.info("Upload complete")
        break
    except Exception as error:
        logger.error(f"Error from Kaggle: {error}")
        time.sleep(5)
        retries += 1

Starting upload for file 1000_cryptos.csv


100%|██████████| 87.9M/87.9M [00:03<00:00, 28.8MB/s]


Upload successful: 1000_cryptos.csv (88MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/ayushkhaire/top-1000-cryptos-historical


# Thank you , and I will see you in next notebook :)